In [1]:
import os
import glob
import numpy as np
import pandas as pd

# Function to clean 2022 retail price data
def clean_retail_price_2022(path):

    # Read and concatenate all retail price files
    df = pd.concat(
        [pd.read_excel(file, usecols=[1, 2, 3, 6, 7]) for file in glob.glob(os.path.join(path, "retail_price_*_2022.xlsx"))],
        ignore_index=True
    )

    # Align column names
    df.columns = ['product_id', 'retail_price', 'currency', 'valid_from', 'valid_to']

    # Set placeholder dates and convert to monthly period
    df[['valid_from', 'valid_to']] = df[['valid_from', 'valid_to']].replace('31.12.9999', '31.12.2050')
    df['valid_from'] = pd.to_datetime(df['valid_from'], format="%d.%m.%Y", errors='coerce').dt.to_period('M')
    df['valid_to'] = pd.to_datetime(df['valid_to'], format="%d.%m.%Y", errors='coerce').dt.to_period('M')

    # Create retail_price_vnd column to ensure all retail prices are in VND
    df['retail_price'] = df['retail_price'].astype('int32')
    df['retail_price_vnd'] = np.where(
        df['currency'] == 'USD',
        np.round(df['retail_price'] * 25000),
        df['retail_price']
    ).astype('int32')

    # Sort and drop duplicate to ensure only the latest-update price is kept
    df = df.sort_values(by=['product_id', 'valid_from', 'valid_to', 'retail_price_vnd'], ascending=True)
    df = df.drop_duplicates(subset=['product_id', 'valid_from'], keep='last')

    # Generate price availability flags for all months in 2022
    months_2022 = pd.period_range('2022-01', '2022-12', freq='M')
    for month in months_2022:
        col_name = str(month)
        df[col_name] = ((df['valid_from'] <= month) & (df['valid_to'] >= month)).astype('int16')

    # Melt the df to get price availability in a long format
    df = df.melt(
        id_vars=['product_id', 'retail_price_vnd'],
        value_vars=[str(m) for m in months_2022],
        var_name='month',
        value_name='price_availability'
    )

    # Keep rows where retail price is available then drop the price_availability column
    df = df[df['price_availability'] == 1].drop(columns=['price_availability'])

    # Drop rows with duplicate product_id and month
    df = df.drop_duplicates(subset=['product_id', 'month'], keep='last')

    # Drop NA values in price_availability column
    df['month'] = df['month'].astype('str').str[5:].astype('int16')

    return df

# Join 2022 sales data with retail price to calculate gross sales amount
def join_sales_with_retail_price(sales_2022_path, retail_price_2022_path):
    
    # Get the sales data of 2022
    sales_2022 = pd.read_csv(sales_2022_path)
    sales_2022['THÁNG2'] = sales_2022['THÁNG'].astype(str).str[4:].astype(int)

    # Get the retail price data of 2022
    retail_price_2022 = clean_retail_price_2022(retail_price_2022_path)

    # Join sales data with retail price
    join_data_2022 = pd.merge(
        sales_2022, 
        retail_price_2022, 
        left_on=['SKU', 'THÁNG2'], 
        right_on=['product_id', 'month'], 
        how='left')

    # Calculate 2022 gross sales amount
    join_data_2022['retail_price_vnd'] = join_data_2022['retail_price_vnd'].fillna(0).astype('int32')
    join_data_2022['TGTT-GTT'] = join_data_2022['retail_price_vnd'] * join_data_2022['SLTT']

    # Align column names
    join_data_2022 = join_data_2022[['MÃ KHÁCH', 'SKU', 'SLTT', 'TGTT-GTT', 'TGTT-GV', 'TGTT-GSCK', 'NĂM', 'THÁNG', 'TUẦN', 'retail_price_vnd']]
    join_data_2022.rename(columns={'SKU':'MÃ HÀNG'}, inplace=True)
    
    return join_data_2022

# Run the functions
sales_2022_path = r'D:\footwear_retail_chain_project\0. input_data\sales\raw_data\sales_raw_2022.csv'
retail_price_2022_path = r'D:\footwear_retail_chain_project\0. input_data\retail_price'
join_data_2022 = join_sales_with_retail_price(sales_2022_path, retail_price_2022_path)

C:\Users\thanh\AppData\Local\Temp\ipykernel_16836\2205292395.py:64: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_2022 = pd.read_csv(sales_2022_path)


In [2]:
# Recheck the retail price processing results with product_id 'DEB008500DEN34'

# Raw price data
path_price = r'D:\footwear_retail_chain_project\0. input_data\retail_price'
raw_price = pd.concat(
        [pd.read_excel(file, usecols=[1, 2, 3, 6, 7]) for file in glob.glob(os.path.join(path_price, "retail_price_*_2022.xlsx"))],
        ignore_index=True
    )
raw_price.columns = ['product_id', 'retail_price', 'currency', 'valid_from', 'valid_to']
raw_price = raw_price[raw_price['product_id'] == 'DEB008500DEN34']
raw_price = raw_price.drop_duplicates(subset=['product_id', 'valid_from', 'valid_to', 'retail_price'], keep='last')
raw_price = raw_price.sort_values(by=['product_id', 'valid_from', 'valid_to', 'retail_price'], ascending=True, inplace=False)
display(raw_price)

# Processed data
processed_price = join_data_2022.copy()
processed_price = processed_price[processed_price['MÃ HÀNG'] == 'DEB008500DEN34'].groupby(by=['MÃ HÀNG', 'THÁNG']).agg(average_price=('retail_price_vnd', 'mean')).reset_index().sort_values(by=['THÁNG'], ascending=True, inplace=False)
display(processed_price)

,product_id,retail_price,currency,valid_from,valid_to
3430629,DEB008500DEN34,305000.0,VND,01.01.2022,13.02.2022
2160319,DEB008500DEN34,305000.0,VND,01.01.2022,31.12.9999
4016607,DEB008500DEN34,305000.0,VND,01.01.2023,31.12.9999
5227614,DEB008500DEN34,325000.0,VND,01.01.2023,31.12.9999
1539489,DEB008500DEN34,280000.0,VND,03.02.2021,31.12.2021
2380083,DEB008500DEN34,299455.0,VND,14.02.2022,14.09.2022
4016606,DEB008500DEN34,299455.0,VND,14.02.2022,31.12.2022
5227613,DEB008500DEN34,319091.0,VND,15.09.2022,31.12.2022


,MÃ HÀNG,THÁNG,average_price
0,DEB008500DEN34,2022001,305000.0
1,DEB008500DEN34,2022002,299455.0
2,DEB008500DEN34,2022003,299455.0
3,DEB008500DEN34,2022004,299455.0
4,DEB008500DEN34,2022005,299455.0
5,DEB008500DEN34,2022006,299455.0
6,DEB008500DEN34,2022007,299455.0
7,DEB008500DEN34,2022008,299455.0
8,DEB008500DEN34,2022009,319091.0
9,DEB008500DEN34,2022010,319091.0


In [3]:
# Compare the result of 2022 with 2023 and 2024

print(f'Sales qty 2022: {join_data_2022["SLTT"].sum()}')
print(f'Gross amount 2022: {join_data_2022["TGTT-GTT"].sum()}')
print(f'Net amount 2022: {join_data_2022["TGTT-GSCK"].sum()}')

sales_2023 = pd.read_csv(r'D:\footwear_retail_chain_project\0. input_data\sales\raw_data\sales_raw_2023.csv')
print(f'Sales qty 2023: {sales_2023["SLTT"].sum()}')
print(f'Gross amount 2023: {sales_2023["TGTT-GTT"].sum()}')
print(f'Net amount 2023: {sales_2023["TGTT-GSCK"].sum()}')

sales_2024 = pd.read_csv(r'D:\footwear_retail_chain_project\0. input_data\sales\raw_data\sales_raw_2024.csv')
print(f'Sales qty 2024: {sales_2024["SLTT"].sum()}')
print(f'Gross amount 2024: {sales_2024["TGTT-GTT"].sum()}')
print(f'Net amount 2024: {sales_2024["TGTT-GSCK"].sum()}')

Sales qty 2022: 8055190
Gross amount 2022: 2569777366486
Net amount 2022: 2277043458782


C:\Users\thanh\AppData\Local\Temp\ipykernel_16836\3594278298.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_2023 = pd.read_csv(r'D:\footwear_retail_chain_project\0. input_data\sales\raw_data\sales_raw_2023.csv')


Sales qty 2023: 6386469
Gross amount 2023: 2114829142804
Net amount 2023: 1869450634284


C:\Users\thanh\AppData\Local\Temp\ipykernel_16836\3594278298.py:12: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_2024 = pd.read_csv(r'D:\footwear_retail_chain_project\0. input_data\sales\raw_data\sales_raw_2024.csv')


Sales qty 2024: 6192119
Gross amount 2024: 1976836210159
Net amount 2024: 1748603616677


In [4]:
# Export to CSV
join_data_2022.to_csv(r'D:\footwear_retail_chain_project\0. input_data\sales\raw_data\sales_raw_2022_adjusted.csv')